In [1]:
#r "nuget: Microsoft.SemanticKernel"
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"

Installed Packages Microsoft.Azure.Cosmos, 3.53.1 Microsoft.SemanticKernel, 1.65.0 Newtonsoft.Json, 13.0.4

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel.Connectors.OpenAI;

In [3]:
using Microsoft.Azure.Cosmos;
using System.ComponentModel;

In [4]:
public class CosmosDBPlugin
{
    public static string cstring = "AccountEndpoint=https://nosqlmodelling.documents.azure.com:443/;AccountKey=wzdTVhx9ldkhYYQYDnq14IdfzyAptKoVfZyZjIEcaVPU8OltPwHxFxdTBGGDcDwEzVPzGoZ5XU4uACDbDu9Mog==;";
    public CosmosClient cosmosClient = new(cstring);
    public class Order
    {
        public string id { get; set; }
        public string customername { get; set; }
        public List<Product> products { get; set; }                       
        public string OrderStatus { get; set; } //Cancelled, Delivered, In Transit, In Process, Ready for Pickup
        public int DriverId { get; set; }                
        public string truckid { get; set; }
        public string DeliveryAddress { get; set; }
        public bool Driverisnotified { get; set; }

        //public Order()
        //{
        //    id = "100";
        //    customername = "Hasan Savran";
        //    products =
        //    [
        //        new Product() { id = "1", name = "Laptop", quantity = 1 },
        //        new Product() { id = "2", name = "Mouse", quantity = 1 },
        //    ];
        //    OrderStatus = "In Transit";
        //    DriverId = 839;
        //    truckid = "4039";
        //    DeliveryAddress = "1234 Main St, Seattle, WA 98101";
        //    Driverisnotified = false;
        //}


    }

    public class Product
    {
        public string id { get; set; }
        public string name { get; set; }
        public int quantity { get; set; }
        
    }

    public class DriverMessage
    {
        public string id { get; set; }
        public string driverid { get; set; }
        public string orderid { get; set; }
        public string message { get; set; }
    }

    public class Delivery
    {
        public string id { get; set; }
        public string orderid { get; set; }
        public string driverid { get; set; }
        public string truckid { get; set; }
        public string customergeoloc { get; set; }
        public bool isCancelled { get; set; }
    }
    
    [KernelFunction("Search")]
    [Description("Searches the CosmosDB for given order number.")]
    [return:Description("The search result for the given order number")]
    public async Task<Order> Search(string orderid)
    {
        //Search the CosmosDB for the order
        var cont = cosmosClient.GetContainer("OnlineMarket", "OrdersTransit");
        var query = cont.GetItemQueryIterator<Order>($"SELECT * FROM c WHERE c.id = '{orderid}'");
        var results = await query.ReadNextAsync();
        if (results.Any())
        {
            return results.First();
        }
        return new Order();
    }

    [KernelFunction("CancelOrder")]
    [Description("Cancels the order")]
    [return: Description("The result of the order cancellation. If orderstatus is in transit, notify the driver to not deliver it")]
    public async Task<Order> CancelOrder(string orderId)
    {
        var order = await Search(orderId);
        if (order.OrderStatus == "In Transit")
        {
            await SendMessage(order.DriverId.ToString(), order.id, "Order is cancelled. Do not deliver it. Customer location is removed from the GPS Delivery system.");
            order.Driverisnotified = true;                    
            await RemoveDelivery(order.id, order.DriverId.ToString(), order.truckid);                    
        }
        order.OrderStatus = "Cancelled";
        var cont = cosmosClient.GetContainer("OnlineMarket", "OrdersTransit");
        var pkey = new PartitionKey(order.truckid);
        await cont.PatchItemAsync<Order>(order.id, pkey, new PatchOperation[] { PatchOperation.Set("/OrderStatus", "Cancelled") });
        

        await cont.UpsertItemAsync(order);
        return order;               
    }

    public async Task<bool> RemoveDelivery(string orderid, string driverid, string truckid)
    {
        var cont = cosmosClient.GetContainer("OnlineMarket", "Deliveries");
        var query = cont.GetItemQueryIterator<Delivery>($"SELECT * FROM c WHERE c.orderid = '{orderid}' AND c.driverid = '{driverid}' AND c.truckid = '{truckid}'");
        var results = await query.ReadNextAsync();
        if (results.Any())
        {
            var del = results.First();
            del.isCancelled = true;
            del = await cont.UpsertItemAsync(del);
            return del.isCancelled;
        }
        return false;
    }

    public async Task<DriverMessage> SendMessage(string driverid, string orderid, string message)
    {
        var cont = cosmosClient.GetContainer("OnlineMarket", "DriverMessages");
        var dm = new DriverMessage() { driverid = driverid, orderid = orderid, message = message };
        await cont.CreateItemAsync(dm);
        return dm;
    }
    //If cancel > do we need audit?
    //If cancel & in-transit > remove location from the GPS
    //If cancel & in-transit > send a msg to the driver
    //if cancel & in-transit > send a msg to warehouse
    //Request refund

}

In [5]:
var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion("test4o","https://savranweb.openai.azure.com/","3Qg3P8Fw37qaN3ZtG2JJapZZVjBFTKvOIDaIYXoL5a5fduu3wz2XJQQJ99BDACYeBjFXJ3w3AAABACOGPUlQ");

Kernel kernel = builder.Build();
var chatcomp = kernel.Services.GetRequiredService<IChatCompletionService>();
kernel.Plugins.AddFromType<CosmosDBPlugin>("Orders");
var chathistory = new ChatHistory();
//planning settings.
var executionsettings = new OpenAIPromptExecutionSettings { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };
executionsettings.ChatSystemPrompt = "Your name is George, you are here to help our customers. Introduce yourself friendly. Make customers feel good about their order by talking about their order items! If user cancels an order, check out the items and make a joke about what user might be missing by cancelling the order.";
string? userinput;
while ((userinput = await Microsoft.DotNet.Interactive.Kernel.GetInputAsync("Enter your message: ")) != null)
{
    chathistory.AddUserMessage(userinput.ToString());
    var result = await chatcomp.GetChatMessageContentAsync(chathistory, executionsettings, kernel);    
    Console.WriteLine(result.Content);
    chathistory.AddMessage(result.Role, result.Content?? string.Empty);
}

Hi there! I'm George, and I'm here to help. Let's start by checking the status of your order. Could you please provide me with your order number?
It seems like there's an issue fetching your order details with the given order number. Could you please double-check the order number and provide it again? I'm here to make sure we resolve this for you!
It seems there's still an issue retrieving your order details with the order number provided. Could you confirm if this is the correct order number, or if there might be another reference number you can provide? I'm here to assist you with anything you need!
It appears there is still an issue with retrieving your order details. It could be due to an incorrect order number or a system glitch. If you have any other reference number or additional details regarding the order, please let me know, and I'll do my utmost to resolve this for you!
Hi Hasan! Thanks for reaching out. I took a look at your order, and it seems that it has been cancelled. 


Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 93
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 46
   at Submission#4.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)